# 01 - Basic Usage of Qurry

In Qurry, we use the instance of `Qurry` to store Quantum Circuit, experiment, build experiment, and excute them by local simulator or pending to the real quantum machine on `IBM`. In this chapter (or we say this jupyter file), we will introduce how to use Qurry to build a simple quantum circuit and measure their Renyi Entropy by local simulator.


If you are using Colab, you can run the following command in terminal to install Qurry.

```bash
pip install -i https://test.pypi.org/simple/ qurry==0.6.10.dev3
```

## 1.1 Setup Environment and Creat experiment executor

In [1]:
from qurry import EntropyMeasure, BackendWrapper
from qiskit import QuantumCircuit

- Before we start, we need to import the package of Qurry and other packages we need. 

- In Qurry, we provider a simple way to import backend called `BackendWrapper`, it will help us to import the backend we need, if your environment includes the GPU acceleration, `qiskit-aer-gpu`, it will be check the availability to access, if does, the GPU backend will also be available.

- The following chapter, we will told you how to use `BackendWrapper` to import the real backend from your IBM account by `IBMProvider` or `AccountProvider`, and the more powerful class `backendManager` to import the real backend more easily.

- In this example, we will use normal `aer_simulator` as our backend


In [2]:
backend = BackendWrapper()
backend('aer')

AerSimulator('aer_simulator_gpu')

In [3]:
print("| Does we have GPU backend to access:", backend.is_aer_gpu)
try:
    # If you have GPU, you can use this backend.
    print(backend('aer_gpu'))
except:
    pass

| Does we have GPU backend to access: True
aer_simulator_gpu


- Now, initialize our `EntropyMeasure`, we have two methods, `randomized` for Randomized Measure as default, and `hadamard` for Hadamard Test, these are our methods for measurement.

- In following tutorial, we will use `randomized` as our measurement method.

In [14]:
experiment_execution_01 = EntropyMeasure(method='hadamard')
print(f'| The first method is which is `Hadamard Test`".')
experiment_execution_02 = EntropyMeasure()
print(f'| The default method is which is `Randomized Measure` we will use.')

| The first method is which is `Hadamard Test`".
| The default method is which is `Randomized Measure` we will use.


---

## 1.2 Load quantum circuit

- We use `TrivialParamagnet` as our target to measure, and add it to our `Qurry` object, and import from `qurry.case` which there are some simple cases.

In [15]:
from qurry.recipe import TrivialParamagnet, GHZ

In [16]:
sample01 = TrivialParamagnet(8)
print("| trivial paramagnet in 8 qubits:")
print(sample01)
sample02 = GHZ(8)
print("| GHZ in 8 qubits:")
print(sample02)
sample03 = QuantumCircuit(8)
sample03.x(range(0,8,2))
print("| Custom circuit:")
print(sample03)

| trivial paramagnet in 8 qubits:
     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     ├───┤
q_5: ┤ H ├
     ├───┤
q_6: ┤ H ├
     ├───┤
q_7: ┤ H ├
     └───┘
| GHZ in 8 qubits:
     ┌───┐                                   
q_0: ┤ H ├──■────────────────────────────────
     └───┘┌─┴─┐                              
q_1: ─────┤ X ├──■───────────────────────────
          └───┘┌─┴─┐                         
q_2: ──────────┤ X ├──■──────────────────────
               └───┘┌─┴─┐                    
q_3: ───────────────┤ X ├──■─────────────────
                    └───┘┌─┴─┐               
q_4: ────────────────────┤ X ├──■────────────
                         └───┘┌─┴─┐          
q_5: ─────────────────────────┤ X ├──■───────
                              └───┘┌─┴─┐     
q_6: ──────────────────────────────┤ X ├──■──
                                   └───┘┌─┴─┐
q_7: ───────────────────────────────────┤ X ├
                        

- After we prepare three quantum circuits, we can add them to our `Qurry` object.
The attribute `.waves` of `Qurry` object is a customized `dict`, we can check it by print it.

In [17]:
experiment_execution_02.add(
    wave=sample01,              # The circuit object
    key='TrivialParamagnet',    # The name of the circuit
    replace=False               # True for if you want to replace the circuit with the same name.
)                               # Default is False, so if you add the same name when False, 
                                # it will be added with serial number.
experiment_execution_02.add(sample02, 'GHZ')
experiment_execution_02.add(sample03)
print(experiment_execution_02.waves)

<WaveContainer={
    TrivialParamagnet: ...
    GHZ: ...
    2: ...
} with 3 waves load, a customized dictionary>


If you do not give the name of circuit, it will be named by series number.

Now, waves are loading, time to excute.

## 1.3 Excute Circuits

- We can use `.measure()` to excute the specific circuit in `EntropyMeasure` object, and it will return hash id of this experiment and save a `Experiment` object in `EntropyMeasure`.
- For `EntropyMeasure`, it will require the following parameters:
  - wave: the name of circuit in `Qurry.waves` object
  - times: the number of random measurement
  - shots: the number of shots for each measurement. Default is 1024

In [18]:
exp1 = experiment_execution_02.measure(
    wave='TrivialParamagnet',
    times=100,
    shots=1024,
)
exp1 

'7be5f60a-1f45-459b-a086-56ba9b16a343'

- And we can check the attribute `.exps` of `EntropyMeasure` object, it will return the hash id of all experiments.

In [19]:
print(experiment_execution_02.exps)

<ExperimentContainer={
    7be5f60a-1f45-459b-a086-56ba9b16a343: ...
} with 1 experiments load, a customized dictionary>


- The following is the other informatiom you can access.

In [31]:
experiment_execution_02.exps[exp1]

<qurrentRandomized.Experiment with exp_id=7be5f60a-1f45-459b-a086-56ba9b16a343, Arguments(exp_name='w=TrivialParamagnet.with100random.qurrent_haar', times=100, measure=(0, 8), unitary_loc=(0, 8), workers_num=16), Commonparams(exp_id='7be5f60a-1f45-459b-a086-56ba9b16a343', wave_key='TrivialParamagnet', shots=1024, backend=GeneralAerSimulator('aer_simulator'), run_args={}, transpile_args={}, tags=(), default_analysis=[], save_location=PosixPath('.'), filename='', files={}, serial=None, summoner_id=None, summoner_name=None, datetimes={'build': '2024-01-09 18:01:38', 'run': '2024-01-09 18:01:39'}), 0 unused arguments, 6 preparing dates, 2 experiment result datasets, and 2 analysis>

In [25]:
print(experiment_execution_02.exps[exp1].statesheet())

 # qurrentRandomized.Experiment with exp_id=7be5f60a-1f45-459b-a086-56ba9b16a343
 - arguments
   - exp_name ---------------- w=TrivialParamagnet.with100random.qurrent_haar
   - times ------------------- 100
   - measure ----------------- (0, 8)
   - unitary_loc ------------- (0, 8)
   - workers_num ------------- 16
 - commonparams
   - exp_id ------------------ 7be5f60a-1f45-459b-a086-56ba9b16a343   # This is ID is generated by Qurry which is different from 'job_id' for pending.
   - wave_key ---------------- TrivialParamagnet
   - shots ------------------- 1024
   - backend ----------------- aer_simulator
   - run_args ---------------- {}
   - transpile_args ---------- {}
   - tags -------------------- ()
   - default_analysis -------- []
   - save_location ----------- .
   - filename ---------------- 
   - files ------------------- {}
   - serial ------------------ None
   - summoner_id ------------- None
   - summoner_name ----------- None
   - datetimes --------------- {'build': '2

- Then, there is a experiment completed, and we can calculate the Renyi Entropy by `analyze`.

---

## 1.4 Result

- We want to know the Renyi Entropy of half system. Then we can use `.analyze()` to calculate it and it will store a `Analysis` object in `Experiment`.
- The following is the parameter `analyze` of `EntropyMeasure` required.
  - degree: The size or range of subsystem, if it is a number, it will be the size of subsystem, if it is a tuple, it will be the range of subsystem.

In [26]:
exp1_report1 = experiment_execution_02.exps[exp1].analyze(
    degree=(0, 4)
)
exp1_report2 = experiment_execution_02.exps[exp1].analyze(
    degree=4
)

|  - 00:00 < ?

|  - 00:00 < ?

- The `analyze` will return the result of this analysis.

In [35]:
print(exp1_report1)
print(exp1_report1.header)

<qurrentRandomized.Analysis with serial=0, AnalysisInput(degree=(0, 4), shots=1024, unitary_loc=(0, 8)), AnalysisContent(purity=1.073450984954834, entropy=-0.10225631753389845, and others), 0 unused arguments>
AnalysisHeader(serial=0, datetime='2024-01-09 18:09:10', summoner=None, log={})


- It can also be found in the attribute `.reports` of `Experiment` object.

In [36]:
print(experiment_execution_02.exps[exp1].reports)
print(experiment_execution_02.exps[exp1].reports[0])
print(experiment_execution_02.exps[exp1].reports[0].header)

<AnalysisContainer={
    0: ...
    1: ...
} with 2 analysis load, a customized dictionary>
<qurrentRandomized.Analysis with serial=0, AnalysisInput(degree=(0, 4), shots=1024, unitary_loc=(0, 8)), AnalysisContent(purity=1.073450984954834, entropy=-0.10225631753389845, and others), 0 unused arguments>
AnalysisHeader(serial=0, datetime='2024-01-09 18:09:10', summoner=None, log={})


- We can use `.export()` to get the result of this analysis.

In [37]:
main, side_prodct = experiment_execution_02.exps[exp1].reports[0].export()

- And there is it

In [38]:
main

{'purity': 1.073450984954834,
 'entropy': -0.10225631753389845,
 'puritySD': 1.0767633054975685,
 'entropySD': 1.447146728472108,
 'bitStringRange': [0, 4],
 'allSystemSource': 'independent',
 'purityAllSys': 1.606309356689453,
 'entropyAllSys': -0.683749766069518,
 'puritySDAllSys': 3.3196231001043306,
 'entropySDAllSys': 2.9814952917981783,
 'bitsStringRangeAllSys': [0, 8],
 'errorRate': -0.26834026997536287,
 'mitigatedPurity': 0.6909323547963591,
 'mitigatedEntropy': 0.5333836233143241,
 'num_qubits': 8,
 'measure': ['measure range:', [0, 8]],
 'measureActually': [0, 8],
 'measureActuallyAllSys': [0, 8],
 'countsNum': 100,
 'takingTime': 0.00121147,
 'takingTimeAllSys': 0.003853926,
 'input': {'degree': [0, 4], 'shots': 1024, 'unitary_loc': [0, 8]},
 'header': {'serial': 0,
  'datetime': '2024-01-09 18:09:10',
  'summoner': None,
  'log': {}}}

- Also, `sideProdct` will record the data of each circuit, and you can use it to do more analysis.

In [39]:
side_prodct

{'purityCells': {95: 0.3610992431640625,
  53: 1.8336105346679688,
  14: 0.7106990814208984,
  93: 0.4074535369873047,
  23: 0.19743728637695312,
  47: 0.3772010803222656,
  82: 0.1747894287109375,
  44: 0.4913616180419922,
  72: 1.3725395202636719,
  0: 0.42932891845703125,
  45: 1.0551433563232422,
  49: 1.5355892181396484,
  62: 0.4603137969970703,
  89: 0.47084808349609375,
  96: 0.4470329284667969,
  81: 0.9820442199707031,
  27: 0.32871246337890625,
  36: 0.8866519927978516,
  56: 0.8186054229736328,
  71: 4.074432373046875,
  84: 1.2966079711914062,
  98: 0.19599533081054688,
  99: 0.3051261901855469,
  15: 0.24875259399414062,
  73: 0.44477272033691406,
  34: 0.9805107116699219,
  69: 0.2294921875,
  80: 1.713510513305664,
  75: 1.6331901550292969,
  29: 0.11350631713867188,
  13: 0.2873992919921875,
  7: 0.5040359497070312,
  55: 1.4826431274414062,
  24: 0.5796699523925781,
  85: 0.17279815673828125,
  91: 1.3730430603027344,
  11: 3.7516517639160156,
  4: 0.4325847625732422,

In [40]:
experiment_execution_02.exps[exp1].reports[0].statesheet()

 # qurrentRandomized.Analysis with serial=0
 - header
   - serial ----------------------- 0
   - datetime --------------------- 2024-01-09 18:09:10
   - summoner --------------------- None
   - log -------------------------- {}
 - input
   - degree ----------------------- (0, 4)
   - shots ------------------------ 1024
   - unitary_loc ------------------ (0, 8)
 - outfields -------------------- 0 ........  # Number of unused arguments.
 - content
   - purity ----------------------- 1.073450984954834
   - entropy ---------------------- -0.10225631753389845
   - puritySD --------------------- 1.0767633054975685
   - entropySD -------------------- 1.447146728472108
   - purityCells ------------------ {95: 0.3610992431640625, 53: 1.8336105346679688, 14: 0.7106990814208984, 93: 0.4074535369873047, 23: 0.19743728637695312, 47: 0.3772010803222656, 82: 0.1747894287109375, 44: 0.4913616180419922, 72: 1.3725395202636719, 0: 0.42932891845703125, 45: 1.0551433563232422, 49: 1.5355892181396484, 62: